In [136]:
import ee
import geemap
import time
import requests
import json
import os
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import geopandas as gpd
import topojson as tp
#ee.Authenticate()
#geemap.ee_search()
ee.Initialize()
Map = geemap.Map()

In [137]:
###token araucaria auth (24feb22)
token='2dVtMj96u9S-NwClUv4unSTaiAae936BjlJEGdX1vdHgzqwX64ehMQK_FEH90UcDPVXYBW5C_7mGmvUTYo2QLLSWT4aNqgT2be_Zf4qESEyXQ86ImRhajNfgYMgMaPA_rxIooqRGm482sw0Gx4IWlPbMD8_rj4iIXUZURaM68vCLArsgBwhibgdpxvQkiiz8JwGM9NoJ1U3ijiKHkhfd2u0suKHpBQ_V6PecmW8LLthNSfq6X8i1BveCVnC5qjPX'
cmpc1='https://araucaria.arauco.com/server/rest/services/Hosted/OTRAS_EMPRESAS_TERCEROS/FeatureServer/0/query?where=objectid+%3E1500+and+empresa%3D%27CMPC%27&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=nom_predio%2C+empresa+&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=&having=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=xyFootprint&resultOffset=&resultRecordCount=&returnTrueCurves=false&returnCentroid=false&sqlFormat=none&resultType=&f=geojson&token='+token
cmpc2='https://araucaria.arauco.com/server/rest/services/Hosted/OTRAS_EMPRESAS_TERCEROS/FeatureServer/0/query?where=objectid+%3C%3D1500+and+empresa%3D%27CMPC%27&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=nom_predio%2C+empresa+&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=&having=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=xyFootprint&resultOffset=&resultRecordCount=&returnTrueCurves=false&returnCentroid=false&sqlFormat=none&resultType=&f=geojson&token='+token
perimetro="data/cmpc_predial_1.json"
df = gpd.read_file(cmpc1)\
        .append(gpd.read_file(cmpc2), ignore_index=True)\
        .to_crs('EPSG:32718')\
        .dissolve(by='empresa')\
        .geometry\
        .simplify(tolerance=5)\
        .to_crs('EPSG:4326')
#df.plot()

In [138]:
t1=2022

In [ ]:
if t1>2016 :
    

In [139]:
if os.path.exists(perimetro):
    os.remove(perimetro)
df.to_file(perimetro, driver="GeoJSON")

In [140]:
gdf = gpd.read_file(
    perimetro
)

In [141]:
fc = geemap.geopandas_to_ee(gdf)

In [142]:
roi = fc\
    .map(lambda f: f.buffer(50))\
    .union()

In [143]:
prefire_start = str(t1-1)+'-09-01'
prefire_end = str(t1-1)+'-11-30'

# Now set the same parameters for AFTER the fire.
postfire_start = str(t1-1)+'-12-01'
#postfire_end = time.strftime('%Y-%m-%d', time.localtime(time.time()))
postfire_end = str(t1)+'-07-31'

In [144]:
area =  ee.FeatureCollection('projects/gl-for-gis-dsr/assets/OrganizationalFeatures/Limite_SPPP_1')
dem = ee.Image('USGS/SRTMGL1_003')
dem_mask=dem.gt(2000)
fires_firms = ee.ImageCollection('FIRMS')\
        .select('T21')\
        .filterDate(prefire_start,postfire_end)\
        .filterBounds(area)

fires_firmsmx = fires_firms.max().clip(area).subtract(273.15)
fires_firmsrc = fires_firmsmx.gte(50)
fires_firmsvc= fires_firmsrc.reduceToVectors(**{
  'geometry':area,
  'crs': 'EPSG:32718',
  'scale':500,
  'geometryType':'polygon',
  'eightConnected': False,
  'bestEffort':False,
  'maxPixels':9e10,
}
  )

fires_firmsvc=ee.FeatureCollection(fires_firmsvc.toList(fires_firmsvc.size()))\
    .filterMetadata('label', 'greater_than', 0)\
    .filterMetadata('count', 'greater_than', 10)
fires_firmsvc = fires_firmsvc.map(lambda f: f.buffer(500))
fires_firmsvc = fires_firmsvc.union()

In [145]:
def addNBR(image):
    nbr = image.normalizedDifference(['B8', 'B12']).rename("nbr")
    image = image.addBands(nbr)
    return image

In [146]:
prefireImCol = ee.ImageCollection('COPERNICUS/S2_SR') \
                  .filterDate(prefire_start, prefire_end) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                  .filterBounds(area) \
                  .map(addNBR)
                  
postfireImCol = ee.ImageCollection('COPERNICUS/S2_SR') \
                  .filterDate(postfire_start, postfire_end) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                  .filterBounds(area) \
                  .map(addNBR)

prefire = prefireImCol.mosaic().clip(fires_firmsvc)
postfire = postfireImCol.mosaic().clip(fires_firmsvc)

In [147]:
preNBR = prefire.select('nbr')
postNBR = postfire.select('nbr')
dNBR_withoutsmooth = preNBR.subtract(postNBR)

dNBR_unscaled = dNBR_withoutsmooth.reduceNeighborhood(
    reducer=ee.Reducer.mean(),
    kernel=ee.Kernel.gaussian(radius=20),
)

#dNBR_unscaled=dNBR_unscaled.clip(r20)
dNBR_reclass = dNBR_unscaled.gte(-0.5).add(dNBR_unscaled.gte(-0.25)).add(dNBR_unscaled.gte(-0.1)).add(dNBR_unscaled.gte(0.1)).add(dNBR_unscaled.gte(0.27)).add(dNBR_unscaled.gte(0.44)).add(dNBR_unscaled.gte(0.66))
#dNBR_Seguro = dNBR_unscaled.gte(0.27)

viz_dNBR={
  'min':1,
  'max':7,#"#7a8737","#acbe4d","#0ae042",
  'palette':["#7a8737","#acbe4d","#0ae042","#fff70b","#ffaf38","#ff641b","#a41fd6"]
}

In [148]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [153]:
def exp(i):    # Define the property name and filter value as strings
    prop_name = 'idzona'
    value = '{}'.format(i)

    # Filter the feature collection based on the attribute condition
    filtered_fc = area.filter(ee.Filter.eq(prop_name, int(value)))

    dNBR_vector= dNBR_reclass.reduceToVectors(**{
    'geometry':filtered_fc,
    'crs': 'EPSG:32718',
    'scale':30,
    'geometryType':'polygon',
    'eightConnected': False,
    'bestEffort':False,
    'maxPixels':9e10,
    }
    )
    dNDBR_export=ee.FeatureCollection(dNBR_vector.toList(dNBR_vector.size()))

    fire=geemap.ee_to_geopandas(dNDBR_export)

    return fire

    # geemap.ee_export_vector_to_drive(
    #     dNDBR_export, description=str(t1)+"_dNBR_500m_"+str(value), fileFormat='GeoJSON', folder="dNBR"
    # )


In [150]:
zonas=['1','2','3','4']

In [155]:
# import necessary modules
import ee
ee.Initialize()

# set study area
aoi = ee.Geometry.Polygon(
  [[[-69.90074204753453, -22.971810371372327],
    [-69.90074204753453,-55.06933517501047],
    [-80.03391717253453,-55.06933517501047],
    [-80.03391717253453,-22.971810371372327]]], None, False)

# filter Landsat 8 SR imagery 
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
              .filterBounds(aoi) \
              .filterDate('2023-01-01', '2023-12-31') \
              .filter(ee.Filter.lt("CLOUD_COVER", 50))

# calculate the differenced Normalized Burn Ratio (dNBR) (eq. 2 & 3)
def calcDNBR(image):
    dnbr = image.normalizedDifference(['B5','B7']) \
                 .subtract(image.normalizedDifference(['B4','B6']))
    return dnbr.rename('dNBR').copyProperties(image, ['system:time_start'])

# apply function to collection and create dNBR composite with median of all images
dnbr = collection.map(calcDNBR).median().clip(aoi)

# mask out areas above 3000 meters in elevation using SRTM elevation dataset
srtm = ee.Image('CGIAR/SRTM90_V4')
elevation_mask = srtm.lt(3000) \
                    .rename('mask') \
                    .float() \
                    .clip(aoi)

dnbr_masked = dnbr.updateMask(elevation_mask)

# display result on Map
Map.centerObject(aoi, 6)
Map.addLayer(dnbr_masked, {'min': -1, 'max': 1, 'palette': ['yellow', 'orange', 'red']}, 'dNBR')
Map.addLayer(elevation_mask, {'opacity': 0.5}, 'Elevation mask')
Map.addLayerControl()
Map


EEException: Image.updateMask: If one image has no bands, the other must also have no bands. Got 0 and 1.

In [154]:
for i in zonas:
    exp(i)

Exception: Computation timed out.